<a href="https://colab.research.google.com/github/lkjh1121/Colab/blob/master/Ch04.%ED%85%8D%EC%8A%A4%ED%8A%B8%20%EB%A7%88%EC%9D%B4%EB%8B%9D%20%EC%8B%A4%EC%8A%B5/%EC%88%98%ED%96%89%ED%8F%89%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
날짜 : 2022/05/27
이름 : 김재현
내용 : 수행평가
"""
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [ ]:
# 데이터 처리함수
def load_data(file):
  result = []

  with open(file, 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

    for line in lines[:5000]:
      data = line.split(',')
      data = data[0] + ' ' + data[1] + '<end>\n'
      result.append(data)

  result = result[1:]
  return result

In [ ]:
# 데이터 불러오기
chat_dataset = load_data('/content/drive/MyDrive/파이썬 데이터 과학실습/file/ChatbotData.csv')
chat_dataset

['12시 땡! 하루가 또 가네요.<end>\n',
 '1지망 학교 떨어졌어 위로해 드립니다.<end>\n',
 '3박4일 놀러가고 싶다 여행은 언제나 좋죠.<end>\n',
 '3박4일 정도 놀러가고 싶다 여행은 언제나 좋죠.<end>\n',
 'PPL 심하네 눈살이 찌푸려지죠.<end>\n',
 'SD카드 망가졌어 다시 새로 사는 게 마음 편해요.<end>\n',
 'SD카드 안돼 다시 새로 사는 게 마음 편해요.<end>\n',
 'SNS 맞팔 왜 안하지ㅠㅠ 잘 모르고 있을 수도 있어요.<end>\n',
 'SNS 시간낭비인 거 아는데 매일 하는 중 시간을 정하고 해보세요.<end>\n',
 'SNS 시간낭비인데 자꾸 보게됨 시간을 정하고 해보세요.<end>\n',
 'SNS보면 나만 빼고 다 행복해보여 자랑하는 자리니까요.<end>\n',
 '가끔 궁금해 그 사람도 그럴 거예요.<end>\n',
 '가끔 뭐하는지 궁금해 그 사람도 그럴 거예요.<end>\n',
 '가끔은 혼자인게 좋다 혼자를 즐기세요.<end>\n',
 '가난한 자의 설움 돈은 다시 들어올 거예요.<end>\n',
 '가만 있어도 땀난다 땀을 식혀주세요.<end>\n',
 '가상화폐 쫄딱 망함 어서 잊고 새출발 하세요.<end>\n',
 '가스불 켜고 나갔어 빨리 집에 돌아가서 끄고 나오세요.<end>\n',
 '가스불 켜놓고 나온거 같아 빨리 집에 돌아가서 끄고 나오세요.<end>\n',
 '가스비 너무 많이 나왔다. 다음 달에는 더 절약해봐요.<end>\n',
 '가스비 비싼데 감기 걸리겠어 따뜻하게 사세요!<end>\n',
 '가스비 장난 아님 다음 달에는 더 절약해봐요.<end>\n',
 '가장 확실한 건 뭘까? 가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.<end>\n',
 '가족 여행 가기로 했어 온 가족이 모두 마음에 드는 곳으로 가보세요.<end>\n',
 '가족 여행 고고 온 가족이 모두 마음에 드는 곳으로 가보세요.<end>\n

In [ ]:
# 텍스트(문장) 생성
text = ' '.join(chat_dataset)
text

'12시 땡! 하루가 또 가네요.<end>\n 1지망 학교 떨어졌어 위로해 드립니다.<end>\n 3박4일 놀러가고 싶다 여행은 언제나 좋죠.<end>\n 3박4일 정도 놀러가고 싶다 여행은 언제나 좋죠.<end>\n PPL 심하네 눈살이 찌푸려지죠.<end>\n SD카드 망가졌어 다시 새로 사는 게 마음 편해요.<end>\n SD카드 안돼 다시 새로 사는 게 마음 편해요.<end>\n SNS 맞팔 왜 안하지ㅠㅠ 잘 모르고 있을 수도 있어요.<end>\n SNS 시간낭비인 거 아는데 매일 하는 중 시간을 정하고 해보세요.<end>\n SNS 시간낭비인데 자꾸 보게됨 시간을 정하고 해보세요.<end>\n SNS보면 나만 빼고 다 행복해보여 자랑하는 자리니까요.<end>\n 가끔 궁금해 그 사람도 그럴 거예요.<end>\n 가끔 뭐하는지 궁금해 그 사람도 그럴 거예요.<end>\n 가끔은 혼자인게 좋다 혼자를 즐기세요.<end>\n 가난한 자의 설움 돈은 다시 들어올 거예요.<end>\n 가만 있어도 땀난다 땀을 식혀주세요.<end>\n 가상화폐 쫄딱 망함 어서 잊고 새출발 하세요.<end>\n 가스불 켜고 나갔어 빨리 집에 돌아가서 끄고 나오세요.<end>\n 가스불 켜놓고 나온거 같아 빨리 집에 돌아가서 끄고 나오세요.<end>\n 가스비 너무 많이 나왔다. 다음 달에는 더 절약해봐요.<end>\n 가스비 비싼데 감기 걸리겠어 따뜻하게 사세요!<end>\n 가스비 장난 아님 다음 달에는 더 절약해봐요.<end>\n 가장 확실한 건 뭘까? 가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.<end>\n 가족 여행 가기로 했어 온 가족이 모두 마음에 드는 곳으로 가보세요.<end>\n 가족 여행 고고 온 가족이 모두 마음에 드는 곳으로 가보세요.<end>\n 가족 여행 어디로 가지? 온 가족이 모두 마음에 드는 곳으로 가보세요.<end>\n 가족 있어? "저를 만들어 준 사람을 부모님<end>\n 가족관계 알려 줘 "저를 만들어

In [ ]:
# 문장 토큰 생성 및 인덱스
tokenizer = Tokenizer(filters='~!@#$%^&*()_+|-=\{}[];:",.?/\t\n')
tokenizer.fit_on_texts([text])
voca_size = len(tokenizer.word_index) + 1

print('전체 토큰 갯수 :', voca_size)
print('토큰 인덱스 :', tokenizer.word_index)

전체 토큰 갯수 : 8915
토큰 인덱스 : {'<end>': 1, '거예요': 2, '너무': 3, '거': 4, '수': 5, '잘': 6, '안': 7, '싶어': 8, '많이': 9, '좋은': 10, '게': 11, '더': 12, '좀': 13, '같아': 14, '있을': 15, '나': 16, '싶다': 17, '해보세요': 18, '것': 19, '같아요': 20, '마세요': 21, '왜': 22, '없어': 23, '있어요': 24, '사람': 25, '다': 26, '드세요': 27, '먹고': 28, '사람이': 29, '오늘': 30, '저도': 31, '하고': 32, '내': 33, '다른': 34, '내가': 35, '같이': 36, '좋죠': 37, '될': 38, '있는': 39, '하세요': 40, '보세요': 41, '뭐': 42, '친구가': 43, '또': 44, '할': 45, '자꾸': 46, '있어': 47, '좋을': 48, '건': 49, '좋아': 50, '돼요': 51, '진짜': 52, '그런': 53, '어떻게': 54, '수도': 55, '그': 56, '것도': 57, '좋아요': 58, '가고': 59, '못': 60, '한': 61, '돈': 62, '없어요': 63, '걸': 64, '가보세요': 65, '제가': 66, '하는': 67, '해': 68, '나만': 69, '있을까': 70, '일이': 71, '열심히': 72, '때': 73, '하지': 74, '하면': 75, '이제': 76, '이렇게': 77, '저는': 78, '있으면': 79, '좋겠다': 80, '될까': 81, '선물': 82, '싫어': 83, '시간': 84, '많아': 85, '스트레스': 86, '마음이': 87, '주세요': 88, '조심하세요': 89, '오세요': 90, '바라요': 91, '맛있게': 92, '말고': 93, '바랍니다': 94, '할까': 95, '술': 96, '안돼': 97, '시

In [ ]:
# 토큰 저장
with open('/content/drive/MyDrive/파이썬 데이터 과학실습/file/chatbot_tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# 문자 인코딩(문자 -> 인덱스 변환)
sequences = []

for line in text.split('\n'):
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
      sequence = encoded[:i+1]
      sequences.append(sequence)

sequences

[[5436, 5437],
 [5436, 5437, 2338],
 [5436, 5437, 2338, 44],
 [5436, 5437, 2338, 44, 2339],
 [5436, 5437, 2338, 44, 2339, 1],
 [5438, 286],
 [5438, 286, 414],
 [5438, 286, 414, 1016],
 [5438, 286, 414, 1016, 1208],
 [5438, 286, 414, 1016, 1208, 1],
 [2340, 777],
 [2340, 777, 17],
 [2340, 777, 17, 1209],
 [2340, 777, 17, 1209, 287],
 [2340, 777, 17, 1209, 287, 37],
 [2340, 777, 17, 1209, 287, 37, 1],
 [2340, 5439],
 [2340, 5439, 777],
 [2340, 5439, 777, 17],
 [2340, 5439, 777, 17, 1209],
 [2340, 5439, 777, 17, 1209, 287],
 [2340, 5439, 777, 17, 1209, 287, 37],
 [2340, 5439, 777, 17, 1209, 287, 37, 1],
 [5440, 2341],
 [5440, 2341, 2342],
 [5440, 2341, 2342, 2343],
 [5440, 2341, 2342, 2343, 1],
 [2344, 5441],
 [2344, 5441, 127],
 [2344, 5441, 127, 483],
 [2344, 5441, 127, 483, 224],
 [2344, 5441, 127, 483, 224, 11],
 [2344, 5441, 127, 483, 224, 11, 180],
 [2344, 5441, 127, 483, 224, 11, 180, 2345],
 [2344, 5441, 127, 483, 224, 11, 180, 2345, 1],
 [2344, 97],
 [2344, 97, 127],
 [2344, 97, 

In [ ]:
# 문장에서 길이가 가장 긴 토큰 갯수
max_len = max(len(l) for l in sequences)
print('max_len : ', max_len)

max_len :  17


In [ ]:
# 패딩처리(모든 토큰 문장의 길이 일치)
paded_sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
paded_sequences

array([[   0,    0,    0, ...,    0, 5436, 5437],
       [   0,    0,    0, ..., 5436, 5437, 2338],
       [   0,    0,    0, ..., 5437, 2338,   44],
       ...,
       [   0,    0,    0, ..., 1512,    5,   15],
       [   0,    0,    0, ...,    5,   15,    2],
       [   0,    0,    0, ...,   15,    2,    1]], dtype=int32)

In [ ]:
# 학습 데이터 라벨 생성
dataset = np.array(paded_sequences)
x_data = dataset[:,:-1]
y_data = dataset[:,-1]

print(x_data[0])
print(y_data[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 5436]
5437


In [ ]:
# 라벨 원-핫 인코딩
y_data = to_categorical(y_data, num_classes=voca_size)
print(y_data[0])

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
# 모델 설계
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(voca_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(voca_size, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# 모델학습
model.fit(x_data, y_data, epochs=50)

Epoch 1/50
1020/1020 [==============================] - 42s 39ms/step - loss: 7.5193 - accuracy: 0.1493
Epoch 2/50
1020/1020 [==============================] - 40s 39ms/step - loss: 6.6970 - accuracy: 0.1551
Epoch 3/50
1020/1020 [==============================] - 48s 47ms/step - loss: 6.3511 - accuracy: 0.1646
Epoch 4/50
1020/1020 [==============================] - 46s 45ms/step - loss: 6.1012 - accuracy: 0.1728
Epoch 5/50
1020/1020 [==============================] - 45s 44ms/step - loss: 5.8671 - accuracy: 0.1791
Epoch 6/50
1020/1020 [==============================] - 44s 43ms/step - loss: 5.6529 - accuracy: 0.1850
Epoch 7/50
1020/1020 [==============================] - 47s 46ms/step - loss: 5.4455 - accuracy: 0.1928
Epoch 8/50
1020/1020 [==============================] - 44s 44ms/step - loss: 5.2489 - accuracy: 0.1997
Epoch 9/50
1020/1020 [==============================] - 45s 44ms/step - loss: 5.0520 - accuracy: 0.2093
Epoch 10/50
1020/1020 [==============================] - 44s 43m

In [ ]:
# 모델 저장
model.save('/content/drive/MyDrive/파이썬 데이터 과학실습/file/chatbot_model.h5')

In [ ]:
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# 토큰 불러오기
with open('/content/drive/MyDrive/파이썬 데이터 과학실습/file/chatbot_tokenizer.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)

tokenizer

In [ ]:
# 모델 불러오기
model = load_model('/content/drive/MyDrive/파이썬 데이터 과학실습/file/chatbot_model.h5')
model

In [ ]:
# 문장 생성 함수
def sentence_generation(current_word):
    init_word = current_word
    sentence = ''

    while True:
        encoded = tokenizer.texts_to_sequences([current_word])
        encoded = pad_sequences([encoded], maxlen=20, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break
        # 종료 체크
        if word == '<end>':
          break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' ' + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + ' --->' + sentence

    return sentence

In [ ]:
# 테스트
print(sentence_generation('12 땡!'))
print(sentence_generation('코 막혀'))
print(sentence_generation('무슨 말을 해야할까'))
print(sentence_generation('안녕'))